In [13]:
import pandas as pd
import requests
import os
from tqdm import tqdm
import time


In [15]:
# --- Configuration ---
INPUT_CSV = '/Users/kristenvinh/Documents/Github_repos/machine_learning_knitting/EDA/Final Sweaters.csv'
OUTPUT_DIR = '/Volumes/Extreme Pro/knitting_photos'
FAILED_LOG = 'failed_downloads.csv' # File to log failed attempts

import pandas as pd
import requests
import os
from tqdm import tqdm
import time
import numpy as np
# --- Configuration ---
BATCH_SIZE = 100

# List of possible extensions to check for resume logic
POSSIBLE_EXTENSIONS = ['.jpg', '.png', '.gif']

MIME_TYPE_MAP = {
    'image/jpeg': '.jpg',
    'image/png': '.png',
    'image/gif': '.gif',
}

# --- Main Script ---
def download_batch(df_chunk, chunk_num):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    batch_failures = []

    for index, row in tqdm(df_chunk.iterrows(), total=df_chunk.shape[0], desc=f"Batch {chunk_num}"):
        sweater_id = row['ID']
        image_url = row['Photo']

        # --- **KEY FIX: ROBUST RESUME LOGIC** ---
        # Check if a file for this ID already exists with ANY of the possible extensions.
        already_exists = False
        for ext in POSSIBLE_EXTENSIONS:
            if os.path.exists(os.path.join(OUTPUT_DIR, f"{sweater_id}{ext}")):
                already_exists = True
                break # Found it, no need to check further
        
        if already_exists:
            continue # Skip to the next image
        # --- End of Fix ---

        try:
            response = requests.get(image_url, headers=headers, timeout=20)
            if response.status_code != 200:
                raise requests.exceptions.HTTPError(f"HTTP Status {response.status_code}")

            content_type = response.headers.get('content-type', '').lower()
            
            file_extension = None
            for mime, ext in MIME_TYPE_MAP.items():
                if mime in content_type:
                    file_extension = ext
                    break

            if not file_extension:
                raise TypeError(f"Unknown image Content-Type: {content_type}")

            output_path = os.path.join(OUTPUT_DIR, f"{sweater_id}{file_extension}")
            
            with open(output_path, 'wb') as f:
                f.write(response.content)

            time.sleep(0.1)

        except (requests.exceptions.RequestException, requests.exceptions.HTTPError, TypeError) as e:
            batch_failures.append({'sweater_id': sweater_id, 'image_url': image_url, 'error': str(e)})

    return batch_failures

def run_batch_downloader():
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    try:
        df = pd.read_csv(INPUT_CSV)
    except FileNotFoundError:
        print(f"Error: Input file '{INPUT_CSV}' not found.")
        return
        
    num_chunks = int(np.ceil(len(df) / BATCH_SIZE))
    list_of_dfs = np.array_split(df, num_chunks)
    
    print(f"Loaded {len(df)} total image links, splitting into {len(list_of_dfs)} batches.")
    all_failed_downloads = []

    for i, chunk_df in enumerate(list_of_dfs):
        chunk_number = i + 1
        print(f"\n--- Processing chunk {chunk_number} of {len(list_of_dfs)} ---")
        failures = download_batch(chunk_df, chunk_number)
        if failures:
            all_failed_downloads.extend(failures)
            print(f"⚠️  {len(failures)} failures in this batch.")
        
    if all_failed_downloads:
        pd.DataFrame(all_failed_downloads).to_csv(FAILED_LOG, index=False)
        print(f"\n--- Process Complete ---")
        print(f"{len(all_failed_downloads)} total images failed. See '{FAILED_LOG}'.")
    else:
        print("\n--- Process Complete ---")
        print("All images downloaded successfully!")

if __name__ == '__main__':
    run_batch_downloader()

/var/folders/4x/ptbsqx910ss5p1lzy1fn39g40000gn/T/ipykernel_99226/2094244286.py:78: DtypeWarning: Columns (5,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(INPUT_CSV)
/opt/anaconda3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Loaded 26022 total image links, splitting into 261 batches.

--- Processing chunk 1 of 261 ---


Batch 1: 100%|██████████| 100/100 [00:01<00:00, 87.40it/s]


⚠️  8 failures in this batch.

--- Processing chunk 2 of 261 ---


Batch 2: 100%|██████████| 100/100 [00:01<00:00, 87.04it/s]


...

⚠️  25 failures in this batch.

--- Processing chunk 20 of 261 ---


Batch 20: 100%|██████████| 100/100 [00:30<00:00,  3.33it/s]


⚠️  12 failures in this batch.

--- Processing chunk 21 of 261 ---


Batch 21:   8%|▊         | 8/100 [00:02<00:27,  3.36it/s]


KeyboardInterrupt: 